In [71]:
import itertools
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils 
from torch.autograd import Variable

from torchvision import transforms

import numpy as np
import pandas as pd

%matplotlib inline

In [100]:
df_label = pd.read_csv('./Platin_Data/Semi_Final/common_patient.csv', sep = '\t', index_col = 0)

In [101]:
df_label

,PLATINUM_STATUS
PATIENT_ID,
TCGA-04-1331,Sensitive
TCGA-04-1332,Sensitive
TCGA-04-1347,Sensitive
TCGA-04-1362,Resistant
TCGA-04-1364,Resistant
...,...
TCGA-61-2098,Sensitive
TCGA-61-2109,Sensitive
TCGA-61-2110,Resistant


In [102]:
df_label.loc[df_label['PLATINUM_STATUS'] == 'Sensitive', 'label'] = 0
df_label.loc[df_label['PLATINUM_STATUS'] == 'Resistant', 'label'] = 1

In [103]:
df_label

,PLATINUM_STATUS,label
PATIENT_ID,,
TCGA-04-1331,Sensitive,0.0
TCGA-04-1332,Sensitive,0.0
TCGA-04-1347,Sensitive,0.0
TCGA-04-1362,Resistant,1.0
TCGA-04-1364,Resistant,1.0
...,...,...
TCGA-61-2098,Sensitive,0.0
TCGA-61-2109,Sensitive,0.0
TCGA-61-2110,Resistant,1.0


In [76]:
df_data = pd.read_csv('./Platin_Data/Semi_Final/PC_tpm_200_minmax.csv', sep = '\t', index_col = 0)

In [13]:
df_data

,TCGA-04-1331,TCGA-04-1332,TCGA-04-1347,TCGA-04-1362,TCGA-04-1364,TCGA-04-1365,TCGA-04-1514,TCGA-04-1530,TCGA-04-1536,TCGA-04-1542,...,TCGA-61-2000,TCGA-61-2008,TCGA-61-2009,TCGA-61-2092,TCGA-61-2097,TCGA-61-2098,TCGA-61-2109,TCGA-61-2110,TCGA-61-2111,TCGA-61-2113
ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000131096,0.001124,0.000566,0.000096,0.002925,0.001883,0.000146,0.000412,0.200155,0.000272,0.000401,...,0.000119,0.000052,0.000138,0.000038,0.000651,0.000309,0.001277,0.000163,0.023144,0.000137
ENSG00000187581,0.074279,0.001385,0.002826,0.016522,1.000000,0.015051,0.017779,0.000000,0.001618,0.018865,...,0.015682,0.001707,0.222856,0.000508,0.005218,0.154442,0.112554,0.006289,0.096016,0.010557
ENSG00000047936,0.149561,0.002034,0.000060,0.002256,0.000016,0.007940,0.000027,0.000705,0.000014,0.000171,...,0.000045,0.000044,0.000303,0.000000,0.000134,0.000215,0.005764,0.000069,0.000012,0.000103
ENSG00000186198,0.020867,0.004467,0.004142,0.005812,0.020709,0.005042,0.001216,0.001000,0.000285,0.001728,...,0.005363,0.001585,0.002821,0.000372,0.003824,0.001345,0.027494,0.000263,0.001709,0.002652
ENSG00000179914,0.008411,0.056473,0.001344,0.001347,0.000372,0.001071,0.001127,0.000742,0.004618,0.000000,...,0.032674,0.001082,0.001927,0.000023,0.006381,0.001390,0.258720,0.030643,0.001957,0.000410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000153822,0.202502,0.002711,0.004158,0.001322,0.000091,0.014324,0.000079,0.004260,0.003334,0.000000,...,0.000672,0.000008,0.000420,0.006529,0.000149,0.000120,0.013265,0.014825,0.000069,0.001336
ENSG00000197616,0.009398,0.000928,0.000315,0.022576,0.002524,0.000320,0.026711,0.000975,0.000014,0.007845,...,0.000700,0.000025,0.000859,0.000000,0.012233,0.010409,0.000838,0.000120,0.002083,0.000067
ENSG00000112499,0.000136,0.000092,0.000016,0.000036,0.000020,0.000008,0.000070,0.000000,0.000004,0.000000,...,0.000058,0.000006,0.000011,0.000011,0.000029,0.000003,0.000723,0.000010,0.000008,0.000017


In [14]:
df_data_mod = df_data.T

In [15]:
df_data_mod

ID,ENSG00000131096,ENSG00000187581,ENSG00000047936,ENSG00000186198,ENSG00000179914,ENSG00000186897,ENSG00000138136,ENSG00000139219,ENSG00000136944,ENSG00000182870,...,ENSG00000048545,ENSG00000188076,ENSG00000048462,ENSG00000187094,ENSG00000106038,ENSG00000153822,ENSG00000197616,ENSG00000112499,ENSG00000111249,ENSG00000203909
TCGA-04-1331,0.001124,0.074279,0.149561,0.020867,0.008411,0.372549,0.000000,0.074310,0.063072,0.002291,...,0.001769,1.000000,0.012713,0.018254,0.000933,0.202502,0.009398,0.000136,0.000482,0.005991
TCGA-04-1332,0.000566,0.001385,0.002034,0.004467,0.056473,0.046285,0.000000,0.005921,0.029110,0.000781,...,0.000264,0.002173,0.000000,0.000000,0.000209,0.002711,0.000928,0.000092,0.000072,0.000000
TCGA-04-1347,0.000096,0.002826,0.000060,0.004142,0.001344,0.793124,0.000461,0.001680,0.000038,0.000037,...,0.000000,0.132969,0.002902,0.000000,0.000000,0.004158,0.000315,0.000016,0.000010,0.224272
TCGA-04-1362,0.002925,0.016522,0.002256,0.005812,0.001347,0.015423,0.003234,0.054898,0.010333,0.001136,...,0.000787,0.000000,0.000000,0.019489,0.000996,0.001322,0.022576,0.000036,0.000539,0.000000
TCGA-04-1364,0.001883,1.000000,0.000016,0.020709,0.000372,0.245882,0.000000,0.010063,0.000123,0.000088,...,0.011812,0.000000,0.001123,0.001152,0.000275,0.000091,0.002524,0.000020,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-61-2098,0.000309,0.154442,0.000215,0.001345,0.001390,0.016812,0.000855,0.233778,0.002315,0.000060,...,0.000050,0.001234,0.000108,0.000442,0.000040,0.000120,0.010409,0.000003,0.001646,0.001015
TCGA-61-2109,0.001277,0.112554,0.005764,0.027494,0.258720,1.000000,0.003060,0.063795,0.004804,0.005207,...,0.000447,0.000000,0.053940,0.001317,0.000000,0.013265,0.000838,0.000723,0.000452,0.000000
TCGA-61-2110,0.000163,0.006289,0.000069,0.000263,0.030643,0.001324,0.000000,0.015528,0.001865,0.008710,...,0.000000,0.000705,0.001476,0.002523,0.000000,0.014825,0.000120,0.000010,0.000007,0.002609
TCGA-61-2111,0.023144,0.096016,0.000012,0.001709,0.001957,0.002696,0.021030,0.000892,0.000157,0.000115,...,0.000033,0.000538,0.003381,0.000000,0.000000,0.000069,0.002083,0.000008,0.000094,0.000000


In [16]:
#df_label_mi = pd.read_csv('./Platin_Data/miRNA_patient.csv', sep = '\t', index_col = 0)

In [17]:
df_data_mi = pd.read_csv('./Platin_Data/Semi_Final/MIR_rpm_100_minmax.csv', sep = '\t', index_col = 0)

In [18]:
df_data_mi = df_data_mi.T

In [9]:
df_data_lnc = pd.read_csv('./Platin_Data/Semi_Final/LNC_tpm_50_minmax.csv', sep = '\t', index_col = 0)

In [10]:
df_data_lnc

,TCGA-04-1331,TCGA-04-1332,TCGA-04-1347,TCGA-04-1362,TCGA-04-1364,TCGA-04-1365,TCGA-04-1514,TCGA-04-1530,TCGA-04-1536,TCGA-04-1542,...,TCGA-61-2000,TCGA-61-2008,TCGA-61-2009,TCGA-61-2092,TCGA-61-2097,TCGA-61-2098,TCGA-61-2109,TCGA-61-2110,TCGA-61-2111,TCGA-61-2113
ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000233048,0.001818,0.000000,0.000000,0.000000,0.008223,0.000284,0.000371,0.000000,0.000000,0.000297,...,0.000000,0.000510,0.000000,0.000000,0.000000,0.000780,0.000031,0.000000,0.000452,0.000000
ENSG00000083622,0.000000,0.001047,0.000000,0.001600,0.000000,0.000581,0.000000,0.003550,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000038,0.000021,0.003009,0.007278
ENSG00000269994,0.000668,0.000547,0.002004,0.002408,0.000000,0.001458,0.000954,0.003711,0.002415,0.000000,...,0.005804,0.001311,0.001049,0.000268,0.000000,0.001336,0.000047,0.000036,0.029038,0.004058
ENSG00000258752,0.222690,0.002003,0.003058,0.003266,0.124308,0.005930,0.529993,0.013590,0.004913,0.000000,...,0.000000,0.053335,0.006401,0.040338,0.001441,0.072042,0.000032,0.000164,0.001181,0.008254
ENSG00000249790,0.003010,0.002467,0.009037,0.001810,0.005150,0.003286,0.021516,0.000000,0.000000,0.000000,...,0.017448,0.011822,0.028378,0.006041,0.014375,0.016571,0.000071,0.000000,0.000000,0.018295
ENSG00000261399,0.566213,0.065518,1.000000,0.040054,0.156729,1.000000,1.000000,1.000000,0.000000,0.000000,...,0.048270,0.327059,0.104678,0.280784,0.053024,0.033341,0.045478,0.004026,0.159373,0.455519
ENSG00000232721,0.000000,0.014051,0.004949,0.079292,0.076157,0.000000,0.377076,0.000000,0.047717,0.007538,...,0.009556,0.246036,0.279755,0.190579,0.090973,0.009901,0.000621,0.000443,0.014341,0.020039
ENSG00000243479,0.004035,0.000661,0.000000,0.046100,0.001726,0.002203,0.011538,0.008974,0.087608,0.027679,...,0.964925,0.063399,0.000000,0.377435,1.000000,0.000000,0.001045,0.000108,0.049149,0.042924
ENSG00000248554,0.088593,0.006224,0.017416,0.021562,0.032485,0.239507,0.126662,0.030493,0.034347,0.040995,...,0.042798,0.020713,0.026518,0.011855,0.010074,0.099242,0.002682,0.000312,0.036703,0.105778


In [11]:
df_data_lnc = df_data_lnc.T

In [12]:
#df_mi = pd.concat([df_label_mi,df_data_mi], axis = 1)

In [13]:
df = pd.concat([df_label, df_data_lnc], axis = 1)
#df = pd.concat([df_label,df_data_mod,df_data_mi,df_data_lnc], axis = 1)

In [14]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(df, test_size = 0.2)

In [15]:
train

,PLATINUM_STATUS,label,ENSG00000233048,ENSG00000083622,ENSG00000269994,ENSG00000258752,ENSG00000249790,ENSG00000261399,ENSG00000232721,ENSG00000243479,...,ENSG00000231621,ENSG00000225472,ENSG00000231764,ENSG00000241388,ENSG00000233536,ENSG00000234692,ENSG00000224717,ENSG00000262117,ENSG00000248646,ENSG00000258170
PATIENT_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-13-0727,Resistant,1.0,0.000300,0.000000,0.001284,0.000000,0.000000,0.358713,0.005072,0.000000,...,0.040832,0.001053,0.000444,0.010228,0.000000,0.015279,0.071954,0.000000,0.000000,0.000000
TCGA-13-0885,Sensitive,0.0,0.000143,0.007784,0.019528,0.000750,0.000831,0.027577,0.001820,0.000000,...,0.029297,0.000302,0.000425,0.000000,0.035835,0.003654,0.000478,0.000000,0.000085,0.003427
TCGA-25-1626,Resistant,1.0,0.000000,0.000755,0.009001,0.009636,0.002136,0.165459,0.009359,0.000000,...,0.301344,0.002721,0.021573,0.001231,0.000000,0.000000,0.012292,0.042849,0.000000,0.007832
TCGA-29-1762,Sensitive,0.0,0.000036,0.000149,0.062580,0.000190,0.002531,0.093347,0.000924,0.003958,...,0.079338,0.003070,0.000377,0.144516,0.002274,0.000000,0.121605,0.000705,0.000000,0.000967
TCGA-13-0726,Sensitive,0.0,0.000112,0.000000,0.000866,0.002348,0.000000,0.129586,0.005701,0.000000,...,0.107078,0.000000,0.000166,0.001000,0.000000,0.008586,0.020217,0.003263,0.000000,0.000298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-61-1914,Sensitive,0.0,0.000000,0.005811,0.000070,0.000535,0.001027,0.004371,0.004500,0.010274,...,0.146755,0.000029,0.000172,0.000379,0.000000,0.006604,0.000591,0.000000,0.000000,0.000000
TCGA-25-1320,Sensitive,0.0,0.000000,0.002702,0.007908,0.075832,0.020374,0.281825,0.033475,0.000000,...,0.509000,0.009268,0.105490,0.001957,0.000000,0.000000,0.064487,0.012772,0.000524,0.044356
TCGA-04-1362,Resistant,1.0,0.000000,0.001600,0.002408,0.003266,0.001810,0.040054,0.079292,0.046100,...,1.000000,0.000000,0.027764,0.002781,0.019518,0.015923,0.072904,0.001513,0.000186,0.002488


In [16]:
train.to_csv("train_platin_total_lnc__50_only.csv", mode='w')

In [17]:
val

,PLATINUM_STATUS,label,ENSG00000233048,ENSG00000083622,ENSG00000269994,ENSG00000258752,ENSG00000249790,ENSG00000261399,ENSG00000232721,ENSG00000243479,...,ENSG00000231621,ENSG00000225472,ENSG00000231764,ENSG00000241388,ENSG00000233536,ENSG00000234692,ENSG00000224717,ENSG00000262117,ENSG00000248646,ENSG00000258170
PATIENT_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-13-0765,Sensitive,0.0,0.000000,0.000000,0.011328,0.010635,0.003929,1.000000,0.068857,0.352923,...,0.369531,0.021447,0.002511,0.003773,0.000000,0.207416,0.004522,0.000000,0.000404,0.001801
TCGA-61-1918,Sensitive,0.0,0.000035,0.000000,0.001640,0.005929,0.000821,0.068171,0.006298,0.004955,...,0.026556,0.000299,0.000053,0.000473,0.000000,0.001807,0.000473,0.000000,0.000085,0.002635
TCGA-30-1860,Sensitive,0.0,0.000000,0.000733,0.000000,0.000749,0.000000,0.009180,0.005452,0.160155,...,0.726590,0.001057,0.000106,0.000159,0.000000,0.000000,0.000477,0.000000,0.000000,0.077565
TCGA-23-2078,Sensitive,0.0,0.000110,0.001576,0.001694,0.004596,0.029288,0.112730,0.005579,0.013658,...,0.284441,0.001622,0.044768,0.001467,0.000000,0.033612,0.394985,0.003193,0.000000,0.001167
TCGA-13-0893,Resistant,1.0,0.000000,0.000000,0.002911,0.052118,0.005251,0.087162,0.011503,0.010560,...,0.246936,0.013376,0.001678,0.002521,0.000000,0.000000,0.000000,0.004389,0.000270,0.016847
TCGA-04-1648,Sensitive,0.0,0.000467,0.008750,0.000857,0.057910,0.000773,0.402176,0.018634,0.002073,...,1.000000,0.000281,0.007810,0.002079,0.002085,0.102054,0.004005,0.000646,0.000000,0.004962
TCGA-61-2109,Sensitive,0.0,0.000031,0.000038,0.000047,0.000032,0.000071,0.045478,0.000621,0.001045,...,0.013329,0.000052,0.000127,0.000014,0.021015,0.000623,0.000489,0.000000,0.000000,0.000000
TCGA-36-1570,Sensitive,0.0,0.000150,0.000000,0.000085,0.069012,0.000771,0.036243,0.005065,0.000000,...,1.000000,0.000105,0.000123,0.000111,0.000000,0.038140,0.000443,0.000322,0.000000,0.000177
TCGA-25-1635,Sensitive,0.0,0.000000,0.000000,0.040385,0.007468,0.016554,0.305311,0.048353,0.110968,...,0.259490,0.027108,0.104405,0.000000,0.000000,0.242752,0.050808,0.000000,0.000000,0.037936


In [18]:
val.to_csv("val_platin_total_lnc__50_only.csv", mode='w')

In [19]:
train = pd.read_csv("train_platin_total_lnc__50_only.csv")
val = pd.read_csv("val_platin_total_lnc__50_only.csv")

In [20]:
trn_X_pd = train.drop(["PATIENT_ID","PLATINUM_STATUS","label"],axis=1).values
trn_y_pd = train.label.values

print("Information of Training Data")
print("Samples : {}".format(trn_X_pd.shape[0]))
print("Features : {}".format(trn_X_pd.shape[1]))

Information of Training Data
Samples : 168
Features : 50


In [21]:
val_X_pd = val.drop(["PATIENT_ID","PLATINUM_STATUS","label"],axis=1).values
val_y_pd = val.label.values

print("Information of Training Data")
print("Samples : {}".format(val_X_pd.shape[0]))
print("Features : {}".format(val_X_pd.shape[1]))

Information of Training Data
Samples : 42
Features : 50


In [22]:
item = []
item_2 = []

print( (val_y_pd == 1.0).sum())
print( (val_y_pd == 0.0).sum())

print( (trn_y_pd == 1.0).sum())
print( (trn_y_pd == 0.0).sum())

13
29
48
120


In [23]:
from sklearn.model_selection import StratifiedKFold

n_splits = 5 # Number of K-fold Splits

splits = list(StratifiedKFold(n_splits=n_splits, shuffle=True).split(trn_X_pd, trn_y_pd))
splits[:5]

[(array([  0,   2,   3,   4,   5,   6,   8,   9,  10,  11,  12,  14,  15,
          16,  17,  18,  19,  20,  22,  23,  24,  25,  26,  27,  30,  31,
          32,  33,  34,  35,  36,  37,  38,  39,  41,  42,  43,  44,  46,
          47,  48,  49,  50,  51,  53,  56,  57,  58,  60,  62,  64,  65,
          67,  68,  69,  70,  71,  72,  73,  76,  77,  78,  81,  82,  83,
          84,  85,  87,  89,  90,  91,  92,  94,  95,  96,  97,  98, 100,
         101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
         114, 115, 116, 117, 118, 120, 121, 123, 125, 128, 130, 131, 132,
         133, 134, 135, 136, 137, 139, 140, 141, 142, 143, 144, 145, 146,
         147, 148, 149, 150, 151, 152, 153, 154, 156, 157, 159, 160, 162,
         163, 164, 165, 166]),
  array([  1,   7,  13,  21,  28,  29,  40,  45,  52,  54,  55,  59,  61,
          63,  66,  74,  75,  79,  80,  86,  88,  93,  99, 119, 122, 124,
         126, 127, 129, 138, 155, 158, 161, 167])),
 (array([  0,   1,   2,   3, 

In [24]:
for i, (train_idx, valid_idx) in enumerate(splits):

    print( (trn_y_pd[valid_idx.astype(int)] == 1.0).sum())
    print( (trn_y_pd[valid_idx.astype(int)] == 0.0).sum())
    
    #print( (trn_y_pd[train_idx.astype(int)] == 1.0).sum())
    #print( (trn_y_pd[train_idx.astype(int)] == 0.0).sum())

10
24
10
24
10
24
9
24
9
24


In [25]:
# for dictionary batch
class Dataset(data_utils.Dataset):
   
    def __init__(self, X, y):
        self.X = X
        self.y = y
   
    def __getitem__(self, idx):
        return {'X': self.X[idx], 'y': self.y[idx]}
   
    def __len__(self):
        return len(self.X)


In [26]:
batch_size = 5

In [27]:
class DNN_seq_1(nn.Module):
    def __init__(self):
        super(DNN_seq_1, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 50, bias=True),
            torch.nn.BatchNorm1d(50),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
        )
        self.classifier2 = nn.Sequential(

            torch.nn.Linear(50, 150, bias=True),
            torch.nn.BatchNorm1d(150),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            

            #torch.nn.Dropout(0.3)
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(150,1, bias=True)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [28]:
#[150,200,100,50]
#[150,200,100,50,20]

class DNN_seq(nn.Module):
    def __init__(self):
        super(DNN_seq, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 200, bias=True),
            torch.nn.BatchNorm1d(200),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
        )
        
        self.classifier2 = nn.Sequential(    
            torch.nn.Linear(200, 300, bias=True),
            torch.nn.BatchNorm1d(300),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
            
        )
        
        self.classifier3 = nn.Sequential(  
            torch.nn.Linear(300, 300, bias=True),
            torch.nn.BatchNorm1d(300),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
            
            torch.nn.Linear(300, 300, bias=True),
            torch.nn.BatchNorm1d(300),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
            
            
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(300, 1, bias=True),
            #nn.Dropout(0.3)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.classifier3(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [129]:
#[150,200,100,50]
#[150,200,100,50,20]

class DNN_seq_2(nn.Module):
    def __init__(self):
        super(DNN_seq_2, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 150, bias=True),
            torch.nn.BatchNorm1d(150),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
        )
        
        self.classifier2 = nn.Sequential(    
            torch.nn.Linear(150, 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
            
        )
        
        self.classifier3 = nn.Sequential(  
            torch.nn.Linear(100, 150, bias=True),
            torch.nn.BatchNorm1d(150),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
            
            torch.nn.Linear(150, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
            
            
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(250, 1, bias=True),
            torch.nn.BatchNorm1d(1)
            #nn.Dropout(0.3)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.classifier3(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [30]:
model = DNN_seq_2()

In [31]:
class CyclicLR(object):
    def __init__(self, optimizer, base_lr=1e-3, max_lr=6e-3,
                 step_size=2000, mode='triangular', gamma=1.,
                 scale_fn=None, scale_mode='cycle', last_batch_iteration=-1):

        if not isinstance(optimizer, Optimizer):
            raise TypeError('{} is not an Optimizer'.format(
                type(optimizer).__name__))
        self.optimizer = optimizer

        if isinstance(base_lr, list) or isinstance(base_lr, tuple):
            if len(base_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} base_lr, got {}".format(
                    len(optimizer.param_groups), len(base_lr)))
            self.base_lrs = list(base_lr)
        else:
            self.base_lrs = [base_lr] * len(optimizer.param_groups)

        if isinstance(max_lr, list) or isinstance(max_lr, tuple):
            if len(max_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} max_lr, got {}".format(
                    len(optimizer.param_groups), len(max_lr)))
            self.max_lrs = list(max_lr)
        else:
            self.max_lrs = [max_lr] * len(optimizer.param_groups)

        self.step_size = step_size

        if mode not in ['triangular', 'triangular2', 'exp_range'] \
                and scale_fn is None:
            raise ValueError('mode is invalid and scale_fn is None')

        self.mode = mode
        self.gamma = gamma

        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = self._triangular_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = self._triangular2_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = self._exp_range_scale_fn
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode

        self.batch_step(last_batch_iteration + 1)
        self.last_batch_iteration = last_batch_iteration

    def batch_step(self, batch_iteration=None):
        if batch_iteration is None:
            batch_iteration = self.last_batch_iteration + 1
        self.last_batch_iteration = batch_iteration
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

    def _triangular_scale_fn(self, x):
        return 1.

    def _triangular2_scale_fn(self, x):
        return 1 / (2. ** (x - 1))

    def _exp_range_scale_fn(self, x):
        return self.gamma**(x)

    def get_lr(self):
        step_size = float(self.step_size)
        cycle = np.floor(1 + self.last_batch_iteration / (2 * step_size))
        x = np.abs(self.last_batch_iteration / step_size - 2 * cycle + 1)

        lrs = []
        param_lrs = zip(self.optimizer.param_groups, self.base_lrs, self.max_lrs)
        for param_group, base_lr, max_lr in param_lrs:
            base_height = (max_lr - base_lr) * np.maximum(0, (1 - x))
            if self.scale_mode == 'cycle':
                lr = base_lr + base_height * self.scale_fn(cycle)
            else:
                lr = base_lr + base_height * self.scale_fn(self.last_batch_iteration)
            lrs.append(lr)
        return lrs

In [32]:
from torch.optim.optimizer import Optimizer

import time # ??

from sklearn.metrics import roc_auc_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc


In [33]:
print(trn_X_pd.shape[0])
          
train_preds = np.zeros(trn_X_pd.shape[0])

## Addiction
train_y_sort = np.zeros(trn_X_pd.shape[0])

test_preds = np.zeros(val_X_pd.shape[0])


train_target = train.label

168


In [34]:
avg_losses_f = []
avg_val_losses_f = []

for i, (train_idx, valid_idx) in enumerate(splits):
    
    ## ???
    x_train_fold = torch.tensor(trn_X_pd[train_idx.astype(int)], dtype=torch.float)   # use_cuse?
    y_train_fold = torch.tensor(trn_y_pd[train_idx.astype(int), np.newaxis], dtype=torch.float)    

    x_val_fold = torch.tensor(trn_X_pd[valid_idx.astype(int)], dtype=torch.float)
    y_val_fold = torch.tensor(trn_y_pd[valid_idx.astype(int), np.newaxis], dtype=torch.float)  
    
################################################################################    
    trn_X = torch.from_numpy(trn_X_pd[train_idx.astype(int)].astype(float))
    trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
    
    # Train
    trn = Dataset(trn_X, trn_y)
    trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True, drop_last = True)  # True or False
    trn_test_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=False, drop_last = False)  # True or False
    
    # Valid
    valid_X = torch.from_numpy(trn_X_pd[valid_idx.astype(int)].astype(float))
    valid_y = torch.from_numpy(trn_y_pd[valid_idx.astype(int)].astype(float))
    
    valid = Dataset(valid_X, valid_y)
    valid_loader = data_utils.DataLoader(valid, batch_size=batch_size, shuffle=False, drop_last = False) # True or False
    
    # Test
    val_X = torch.from_numpy(val_X_pd.astype(float))
    val_y = torch.from_numpy(val_y_pd.astype(float))
    val = Dataset(val_X, val_y)
    test_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)
   ################################################################################    
    print(i)
    
    ## Loss function
    criterion = nn.BCELoss()
    learning_rate = 1e-3
    
    ## After just second, will modify optimizer(weight_decay,step_size, base_lr, max_lr,scheduler)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay = 1e-5)
    
    model = DNN_seq_1()
    use_cuda = torch.cuda.is_available()
    num_epochs = 50
    
    if use_cuda:
        model = model.cuda()
    
    ##############################################################################
    step_size = 2000
    base_lr, max_lr = 0.001, 0.01  
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                             lr=max_lr)
    
    scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, step_size=step_size, mode='exp_range', gamma=0.99994)    
    
    ##############################################################################
    
    print(f'Fold {i+1}')
    
    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        avg_loss = 0.
        #correct = 0.   # Accuracy
        
        for batch_idx, trn in enumerate(trn_loader):
            trn_X, trn_y = trn['X'], trn['y']
            if use_cuda:
                trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
            trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
            optimizer.zero_grad()
            trn_pred = model(trn_X)

            
            if scheduler:
                #print('cycle_LR')
                scheduler.batch_step()
            #print(trn_pred.squeeze())
            #print(trn_y)
            trn_loss = criterion(trn_pred.squeeze(), trn_y)
            trn_loss.backward()
            optimizer.step()
        
            avg_loss += trn_loss.item()/len(trn_loader)
        
            #predicted = torch.max(trn_pred.data, 1)[1] 
            #correct += (predicted == trn_y).sum()
        model.eval()
        
        valid_preds_fold = np.zeros((valid_X.size(0)))
        test_preds_fold = np.zeros(val_X_pd.shape[0])  # Test
        
        avg_val_loss = 0.
        
        with torch.no_grad():
            for batch_idx, val in enumerate(valid_loader):
                val_X, val_y = val['X'], val['y']
                if use_cuda:
                    val_X, val_y = val_X.cuda(), val_y.cuda()
                val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
                optimizer.zero_grad()
                val_pred = model(val_X).detach()

            
                val_loss = criterion(val_pred.squeeze(), val_y)
        
                avg_val_loss += val_loss.item()/len(valid_loader)
            
            
                #val_pred = torch.max(val_pred, 1)[1]
                val_pred = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
                #print(val_pred.cpu().numpy())
                #print(val_pred.cpu().numpy()[:,0])
            
                valid_preds_fold[batch_idx * batch_size:(batch_idx+1) * batch_size] = (val_pred.cpu().numpy())    # modified [:,0]
            
            #valid_preds_fold_2 = 
                # Loss function chage -> plus Sigmoid
            elapsed_time = time.time() - start_time 
            print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t time={:.2f}s'.format(epoch + 1, num_epochs, avg_loss, avg_val_loss, elapsed_time))            

# Test        
    avg_losses_f.append(avg_loss)
    avg_val_losses_f.append(avg_val_loss) 
    
    for batch_idx, test in enumerate(test_loader):
        test_X, test_y = test['X'], test['y']
        if use_cuda:
            test_X, test_y = test_X.cuda(), test_y.cuda()
        test_X, test_y = Variable(test_X).float(), Variable(test_y).float()        
        test_pred = model(test_X).detach()
        
        
        test_pred = (test_pred > 0.5).flatten().type(torch.ByteTensor)
        
        #print(test_pred)
        #print(test_pred.cpu().numpy())
        test_preds_fold[batch_idx * batch_size:(batch_idx+1) * batch_size] = (test_pred.cpu().numpy())   # modified [:,0]
        
        
    train_preds[valid_idx.astype(int)] = valid_preds_fold
    print(valid_preds_fold)
    print(trn_y_pd[valid_idx.astype(int)])
    train_y_sort[valid_idx.astype(int)] = trn_y_pd[valid_idx.astype(int)]
    test_preds += test_preds_fold / len(splits)

#predict_ = pd.DataFrame(predict)

#predict_.iloc[:,0]

#label = pd.DataFrame(val_y_pd)

#test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)    
    
print(trn_y_pd)
print(train_preds)

auc  =  round(roc_auc_score(train_y_sort,train_preds.astype(int)),4)


cnf = confusion_matrix(train_y_sort, train_preds, labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

print('AUC: ', auc)


#fpr, trp, _  =  roc_auc_score(train_y_sort,train_preds.astype(int))

#fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

#print(type(fpr))
#print(fpr)

#auc = auc(fpr, trp)
#print('AUC: ', auc(fpr, trp))
print('\n')
print('All \t loss={:.4f} \t val_loss={:.4f} \t auc={:.4f}'.format(np.average(avg_losses_f),np.average(avg_val_losses_f),auc))        
        

0
Fold 1
Epoch 1/50 	 loss=0.6270 	 val_loss=0.6313 	 time=0.13s
Epoch 2/50 	 loss=0.5605 	 val_loss=0.5957 	 time=0.10s
Epoch 3/50 	 loss=0.5530 	 val_loss=0.5671 	 time=0.09s
Epoch 4/50 	 loss=0.5520 	 val_loss=0.5545 	 time=0.08s
Epoch 5/50 	 loss=0.5468 	 val_loss=0.5319 	 time=0.11s
Epoch 6/50 	 loss=0.5232 	 val_loss=0.5100 	 time=0.08s
Epoch 7/50 	 loss=0.4973 	 val_loss=0.4909 	 time=0.08s
Epoch 8/50 	 loss=0.4986 	 val_loss=0.4906 	 time=0.12s
Epoch 9/50 	 loss=0.5061 	 val_loss=0.4942 	 time=0.11s
Epoch 10/50 	 loss=0.5144 	 val_loss=0.5283 	 time=0.09s
Epoch 11/50 	 loss=0.4587 	 val_loss=0.5283 	 time=0.13s
Epoch 12/50 	 loss=0.4839 	 val_loss=0.5354 	 time=0.18s
Epoch 13/50 	 loss=0.4943 	 val_loss=0.5300 	 time=0.15s
Epoch 14/50 	 loss=0.4680 	 val_loss=0.5455 	 time=0.12s
Epoch 15/50 	 loss=0.4661 	 val_loss=0.5570 	 time=0.08s
Epoch 16/50 	 loss=0.4920 	 val_loss=0.5430 	 time=0.11s
Epoch 17/50 	 loss=0.4455 	 val_loss=0.5386 	 time=0.09s
Epoch 18/50 	 loss=0.4761 	 val

Epoch 39/50 	 loss=0.3913 	 val_loss=0.6761 	 time=0.10s
Epoch 40/50 	 loss=0.4219 	 val_loss=0.6139 	 time=0.09s
Epoch 41/50 	 loss=0.3914 	 val_loss=0.7047 	 time=0.08s
Epoch 42/50 	 loss=0.4225 	 val_loss=0.6847 	 time=0.09s
Epoch 43/50 	 loss=0.3558 	 val_loss=0.6972 	 time=0.09s
Epoch 44/50 	 loss=0.3295 	 val_loss=0.7246 	 time=0.09s
Epoch 45/50 	 loss=0.4583 	 val_loss=0.6038 	 time=0.08s
Epoch 46/50 	 loss=0.4023 	 val_loss=0.5514 	 time=0.09s
Epoch 47/50 	 loss=0.3837 	 val_loss=0.7223 	 time=0.10s
Epoch 48/50 	 loss=0.3727 	 val_loss=0.6101 	 time=0.11s
Epoch 49/50 	 loss=0.4034 	 val_loss=0.5957 	 time=0.10s
Epoch 50/50 	 loss=0.2889 	 val_loss=0.5924 	 time=0.09s
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1.
 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
[0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 1. 0. 1. 0. 1. 0.]
3
Fold 4
Epoch 1/50 	 loss=0.6970 	 val_loss=0.6257 	 time=0.12s
Epoch 2/50 	 loss=0.5828 	 val_loss=0.533

# Valid

In [66]:
model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(valid_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        #print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        print(val_y)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(valid_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(valid_loader)*batch_size)))

tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 0.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([1., 1., 1., 0., 0.])
tensor([0, 0, 0, 1, 0], dtype=torch.uint8)
tensor([1., 0., 0., 0., 0.])
tensor([0, 0, 1, 0, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 0.])
tensor([1, 1, 1, 0, 0], dtype=torch.uint8)
tensor([1., 1., 1., 0., 0.])
tensor([1, 1, 1, 0, 0], dtype=torch.uint8)
tensor([1., 0., 1., 0., 0.])
tensor([0, 0, 1], dtype=torch.uint8)
tensor([0., 0., 0.])
******************** Test ********************
Loss: 0.30895769596099854, Accuracy: 0.7142857313156128 %
**********************************************
Val accuracy:0.714


In [67]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

pd.DataFrame(trn_y_pd[valid_idx.astype(int)], predict_.iloc[:,0])

,0
0,
6.080831e-15,0.0
3.896444e-01,0.0
1.800919e-01,0.0
1.123364e-01,0.0
4.831288e-01,0.0
2.254827e-01,1.0
2.801564e-02,1.0
7.484800e-02,1.0
3.731060e-06,0.0


In [68]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(trn_y_pd[valid_idx.astype(int)])

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

#fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])
auc  =  round(roc_auc_score(test_p['label'],test_p['predicted_prob']),4)
#print(fpr)
#print(trp)
#print(fpr)
print('AUC: ', auc)

[[ 5  4]
 [ 4 20]]
Accuracy :  0.7575757575757576
Sensitivity :  0.5555555555555556
Specificity :  0.8333333333333334
AUC:  0.6944


In [69]:
len(x_train_fold)

135

In [70]:
model = DNN_seq_1()

# Model Load

In [177]:
#model = TheModelClass(*args, **kwargs)                                                                                                     
#optimizer = TheOptimizerClass(*args, **kwargs)                                                                                             
checkpoint = torch.load("./platin_model_save/platin_model_300_100_100_model_1_150_250.pth")                                                                                  
model.load_state_dict(checkpoint['model_state_dict'])                                                                                         
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])                                                                               
#num_epochs = checkpoint['epoch']                                                                                                           
loss = checkpoint['loss']                                                                                                                   
model.eval()  

DNN_seq_1(
  (classifier): Sequential(
    (0): Linear(in_features=500, out_features=150, bias=True)
    (1): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
  (classifier2): Sequential(
    (0): Linear(in_features=150, out_features=250, bias=True)
    (1): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
  (output_layer): Sequential(
    (0): Linear(in_features=250, out_features=1, bias=True)
  )
)

# Test

In [71]:
model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        #print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        print(val_y)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(test_loader)*batch_size)))

tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 0., 1.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 0., 0.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 0., 1., 0., 0.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([1., 0., 1., 0., 0.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 1., 0.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 1.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 0., 0.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 0., 1., 0., 1.])
tensor([0, 0], dtype=torch.uint8)
tensor([1., 1.])
******************** Test ********************
Loss: 0.7104251384735107, Accuracy: 0.6222222447395325 %
**********************************************
Val accuracy:0.622


In [72]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
0.493109,0.0
0.493315,1.0
0.490894,0.0
0.493182,0.0
0.492600,1.0
0.491756,0.0
0.492351,1.0
0.491634,0.0
0.491395,0.0


In [73]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(val_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print(type(fpr))
#print(fpr)
print('AUC: ', auc(fpr, trp))

[[ 0 14]
 [ 0 28]]
Accuracy :  0.6666666666666666
Sensitivity :  0.0
Specificity :  1.0
<class 'numpy.ndarray'>
AUC:  0.5


In [181]:
torch.save({
#'epoch': EPOCHS,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': trn_loss
    }, "./platin_model_save/platin_model_300_100_100_model_1_150_250___.pth")     

# Train Test (Except Valid)

In [74]:
model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(trn_test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(trn_test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(trn_test_loader)*batch_size)))

tensor([[0.4910],
        [0.4930],
        [0.4933],
        [0.4881],
        [0.4925]])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[0.4888],
        [0.4965],
        [0.4882],
        [0.4894],
        [0.4916]])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[0.4920],
        [0.4901],
        [0.4933],
        [0.4892],
        [0.4921]])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[0.4914],
        [0.4929],
        [0.4918],
        [0.4873],
        [0.4906]])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[0.4916],
        [0.4917],
        [0.4909],
        [0.4888],
        [0.4924]])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[0.4891],
        [0.4934],
        [0.4927],
        [0.4933],
        [0.4929]])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[0.4911],
        [0.4915],
        [0.4918],
        [0.4935],
        [0.4904]])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[0.4920],
        [0.4916],
        [0.4919],
        

In [75]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]
#trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
pd.DataFrame(trn_y_pd[train_idx.astype(int)], predict_.iloc[:,0])

,0
0,
0.490963,1.0
0.492982,0.0
0.493317,0.0
0.488129,0.0
0.492504,0.0
...,...
0.492123,1.0
0.492540,0.0
0.491913,0.0


In [63]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(trn_y_pd[train_idx.astype(int)])

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print('AUC: ', auc(fpr, trp))

[[38  0]
 [97  0]]
Accuracy :  0.2814814814814815
Sensitivity :  1.0
Specificity :  0.0
AUC:  0.5


# .........................................................................

# Real Train stage

# .........................................................................

# Train (Total Data)

In [35]:
trn_X = torch.from_numpy(trn_X_pd.astype(float))
trn_y = torch.from_numpy(trn_y_pd.astype(float))

val_X = torch.from_numpy(val_X_pd.astype(float))
val_y = torch.from_numpy(val_y_pd.astype(float))

In [36]:
trn = Dataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True, drop_last = True)

trn_test_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=False)


val = Dataset(val_X, val_y)
val_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)

In [37]:
## Loss function
criterion = nn.BCELoss()
learning_rate = 1e-3
    
## After just second, will modify optimizer(weight_decay,step_size, base_lr, max_lr,scheduler)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay = 1e-5)
    
model = DNN_seq_2()
use_cuda = torch.cuda.is_available()
num_epochs = 90
    
if use_cuda:
    model = model.cuda()
    
##############################################################################
step_size = 2000
base_lr, max_lr = 0.001, 0.01  
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                             lr=max_lr)
    
scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, step_size=step_size, mode='exp_range', gamma=0.99994)    

In [38]:


predict = []

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    avg_loss = 0.
    #correct = 0.   # Accuracy
    correct_val = 0.
        
    for batch_idx, trn in enumerate(trn_loader):
        trn_X, trn_y = trn['X'], trn['y']
        if use_cuda:
            trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
        trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
        optimizer.zero_grad()
        trn_pred = model(trn_X)
        for i in trn_pred:
            predict.append(i.detach().numpy())
            
        if scheduler:
            #print('cycle_LR')
            scheduler.batch_step()
        #print(trn_pred.squeeze())
        #print(trn_y)
        trn_loss = criterion(trn_pred.squeeze(), trn_y)
        trn_loss.backward()
        optimizer.step()
        
        avg_loss += trn_loss.item()/len(trn_loader)
        elapsed_time = time.time() - start_time
        predicted = (trn_pred >= 0.5).flatten().type(torch.ByteTensor)
        correct_val += (predicted == trn_y).sum()
    print('*'*20, 'Train', '*'*20)
    print('Loss: {}, Accuracy: {} % \t time={:.2f}s'.format(trn_loss.item(), correct_val/(len(trn_loader)*batch_size), elapsed_time))
print('*'*46)
print("Trn accuracy:{:.3f} ".format(float(correct_val) / (len(trn_loader)*batch_size)))

******************** Train ********************
Loss: 1.0370371341705322, Accuracy: 0.49696969985961914 % 	 time=0.34s
******************** Train ********************
Loss: 0.4805886149406433, Accuracy: 0.539393961429596 % 	 time=0.29s
******************** Train ********************
Loss: 0.7496358156204224, Accuracy: 0.6121212244033813 % 	 time=0.28s
******************** Train ********************
Loss: 0.8888704180717468, Accuracy: 0.5696969628334045 % 	 time=0.28s
******************** Train ********************
Loss: 0.4830182194709778, Accuracy: 0.6545454263687134 % 	 time=0.30s
******************** Train ********************
Loss: 1.032348394393921, Accuracy: 0.6242424249649048 % 	 time=0.32s
******************** Train ********************
Loss: 0.5486370325088501, Accuracy: 0.6727272868156433 % 	 time=0.30s
******************** Train ********************
Loss: 0.46035242080688477, Accuracy: 0.6727272868156433 % 	 time=0.28s
******************** Train ********************
Loss: 0.

******************** Train ********************
Loss: 0.4314555525779724, Accuracy: 0.7636363506317139 % 	 time=0.30s
******************** Train ********************
Loss: 0.6116787791252136, Accuracy: 0.7272727489471436 % 	 time=0.32s
******************** Train ********************
Loss: 0.21853002905845642, Accuracy: 0.7515151500701904 % 	 time=0.28s
******************** Train ********************
Loss: 0.43295758962631226, Accuracy: 0.8060606122016907 % 	 time=0.31s
******************** Train ********************
Loss: 0.2347404509782791, Accuracy: 0.8363636136054993 % 	 time=0.31s
******************** Train ********************
Loss: 0.18524721264839172, Accuracy: 0.8181818127632141 % 	 time=0.32s
******************** Train ********************
Loss: 0.3621549606323242, Accuracy: 0.8121212124824524 % 	 time=0.35s
******************** Train ********************
Loss: 0.2180204689502716, Accuracy: 0.7696969509124756 % 	 time=0.31s
******************** Train ********************
Loss:

In [39]:
model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(trn_test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(trn_test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(trn_test_loader)*batch_size)))

tensor([[0.3974],
        [0.1855],
        [0.2668],
        [0.0728],
        [0.2247]])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[0.0610],
        [0.4087],
        [0.4424],
        [0.1793],
        [0.2813]])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[0.1072],
        [0.0815],
        [0.4435],
        [0.1815],
        [0.1593]])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[0.0377],
        [0.0361],
        [0.0371],
        [0.3779],
        [0.1261]])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[0.5739],
        [0.1788],
        [0.1742],
        [0.2784],
        [0.1174]])
tensor([1, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[0.2394],
        [0.5477],
        [0.2490],
        [0.0290],
        [0.1413]])
tensor([0, 1, 0, 0, 0], dtype=torch.uint8)
tensor([[0.2047],
        [0.2901],
        [0.1648],
        [0.2002],
        [0.4745]])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[0.2104],
        [0.2634],
        [0.1128],
        

In [40]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]
#trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
pd.DataFrame(trn_y_pd, predict_.iloc[:,0])

,0
0,
0.397393,1.0
0.185460,0.0
0.266760,1.0
0.072809,0.0
0.224691,0.0
...,...
0.174780,0.0
0.068231,0.0
0.413631,1.0


In [41]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(trn_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print('AUC: ', auc(fpr, trp))

[[ 19  29]
 [  0 120]]
Accuracy :  0.8273809523809523
Sensitivity :  0.3958333333333333
Specificity :  1.0
AUC:  0.6979166666666666


# Final Test

In [42]:
model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        #print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        print(val_y)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(test_loader)*batch_size)))

tensor([0, 0, 1, 0, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 1.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 1.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 0., 1., 0., 1.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([1., 0., 0., 0., 1.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 0.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([1., 0., 0., 0., 1.])
tensor([0, 1, 0, 0, 1], dtype=torch.uint8)
tensor([0., 0., 1., 1., 1.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 0., 1., 0., 0.])
tensor([1, 0], dtype=torch.uint8)
tensor([1., 0.])
******************** Test ********************
Loss: 0.3967731297016144, Accuracy: 0.644444465637207 %
**********************************************
Val accuracy:0.644


In [43]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
0.449398,0.0
0.173810,0.0
0.525002,0.0
0.017654,0.0
0.318993,1.0
0.408329,0.0
0.186568,0.0
0.449737,0.0
0.174035,0.0


In [44]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(val_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print(type(fpr))
#print(fpr)
print('AUC: ', auc(fpr, trp))

[[ 2 11]
 [ 2 27]]
Accuracy :  0.6904761904761905
Sensitivity :  0.15384615384615385
Specificity :  0.9310344827586207
<class 'numpy.ndarray'>
AUC:  0.5424403183023871


# Model Save

# require to check~~

In [74]:
torch.save({
#'epoch': EPOCHS,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': trn_loss
    }, "./platin_model_save/platin_model_200_100_100_model_1_100_250__1.pth")     

# Transfer Learning

결론적으로 이미 잘 훈련된 모델이 있고, 특히 해당 모델과 유사한 문제를 해결시 transfer learining을 사용합니다.
실질적 조언
새로 훈련할 데이터가 적지만 original 데이터와 유사할 경우

데이터의 양이 적어 fine-tune (전체 모델에 대해서 backpropagation을 진행하는 것) 은 over-fitting의 위험이 있기에 하지 않습니다.
새로 학습할 데이터는 original 데이터와 유사하기 때문에 이 경우 최종 linear classfier 레이어만 학습을 합니다.
새로 훈련할 데이터가 매우 많으며 original 데이터와 유사할 경우

새로 학습할 데이터의 양이 많다는 것은 over-fitting의 위험이 낮다는 뜻이므로, 전체 레이어에 대해서 fine-tune을 합니다.
새로 훈련할 데이터가 적으며 original 데이터와 다른 경우

데이터의 양이 적기 때문에 최종 단계의 linear classifier 레이어를 학습하는 것이 좋을 것입니다. 반면서 데이터가 서로 다르기 때문에 거의 마지막부분 (the top of the network)만 학습하는 것은 좋지 않습니다. 서로 상충이 되는데.. 이 경우에는 네트워크 초기 부분 어딘가 activation 이후에 특정 레이어를 학습시키는게 좋습니다.
새로 훈련할 데이터가 많지만 original 데이터와와 다른 경우

데이터가 많기 때문에 아예 새로운 ConvNet을 만들수도 있지만, 실적적으로 transfer learning이 더 효율이 좋습니다. 전체 네트워크에 대해서 fine-tune을 해도 됩니다.

In [45]:
model.classifier3

Sequential(
  (0): Linear(in_features=100, out_features=150, bias=True)
  (1): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=150, out_features=250, bias=True)
  (5): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): Dropout(p=0.5, inplace=False)
)

In [130]:
#[150,200,100,50]
#[150,200,100,50,20]

class DNN_seq_2(nn.Module):
    def __init__(self):
        super(DNN_seq_2, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 150, bias=True),
            torch.nn.BatchNorm1d(150),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
        )
        
        self.classifier2 = nn.Sequential(    
            torch.nn.Linear(150, 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
            
        )
        
        self.classifier3 = nn.Sequential(  
            torch.nn.Linear(100, 150, bias=True),
            torch.nn.BatchNorm1d(150),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
            
            torch.nn.Linear(150, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
            
            
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(250, 1, bias=True),
            torch.nn.BatchNorm1d(1)
            #nn.Dropout(0.3)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.classifier3(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [131]:
model_2 = DNN_seq_2()

In [132]:
for param in model.parameters():
    param.requires_grad = True

model.classifier = model_2.classifier    
model.classifier2 = model_2.classifier2


In [133]:
df_data = pd.read_csv('./Platin_Data/Semi_Final/PC_tpm_200_minmax.csv', sep = '\t', index_col = 0)

In [134]:
df_data_mod = df_data.T

In [135]:
df_data_mi = pd.read_csv('./Platin_Data/Semi_Final/MIR_rpm_100_minmax.csv', sep = '\t', index_col = 0)

In [136]:
df_data_mi = df_data_mi.T

In [137]:
df = pd.concat([df_label,df_data_mod,df_data_mi], axis = 1)

In [138]:
df

,PLATINUM_STATUS,label,ENSG00000131096,ENSG00000187581,ENSG00000047936,ENSG00000186198,ENSG00000179914,ENSG00000186897,ENSG00000138136,ENSG00000139219,...,hsa-mir-514a-1,hsa-mir-101-2,hsa-mir-6892,hsa-mir-514a-2,hsa-mir-331,hsa-mir-3651,hsa-mir-550a-3,hsa-mir-6884,hsa-mir-1224,hsa-mir-3620
PATIENT_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-04-1331,Sensitive,0.0,0.001124,0.074279,0.149561,0.020867,0.008411,0.372549,0.000000,0.074310,...,0.000000,0.013312,0.006381,0.000037,0.008103,0.000593,0.000886,0.000332,0.001581,0.000252
TCGA-04-1332,Sensitive,0.0,0.000566,0.001385,0.002034,0.004467,0.056473,0.046285,0.000000,0.005921,...,0.000000,0.004477,0.002428,0.000000,0.007846,0.002270,0.005067,0.000000,0.001556,0.000147
TCGA-04-1347,Sensitive,0.0,0.000096,0.002826,0.000060,0.004142,0.001344,0.793124,0.000461,0.001680,...,0.000000,0.007213,0.007008,0.000000,0.017185,0.001530,0.001314,0.000216,0.000000,0.000357
TCGA-04-1362,Resistant,1.0,0.002925,0.016522,0.002256,0.005812,0.001347,0.015423,0.003234,0.054898,...,0.020890,0.014762,0.002817,0.016416,0.066030,0.001828,0.002836,0.000479,0.004774,0.003333
TCGA-04-1364,Resistant,1.0,0.001883,1.000000,0.000016,0.020709,0.000372,0.245882,0.000000,0.010063,...,0.003523,0.006381,0.015363,0.004760,0.067332,0.000710,0.000300,0.000252,0.001199,0.000942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-61-2098,Sensitive,0.0,0.000309,0.154442,0.000215,0.001345,0.001390,0.016812,0.000855,0.233778,...,0.009848,0.003453,0.000070,0.005062,0.005220,0.000150,0.000351,0.000014,0.000179,0.000072
TCGA-61-2109,Sensitive,0.0,0.001277,0.112554,0.005764,0.027494,0.258720,1.000000,0.003060,0.063795,...,0.000188,0.012297,0.005054,0.000223,0.002850,0.000077,0.000069,0.000009,0.000158,0.000365
TCGA-61-2110,Resistant,1.0,0.000163,0.006289,0.000069,0.000263,0.030643,0.001324,0.000000,0.015528,...,0.001060,0.014861,0.001302,0.000619,0.004408,0.000117,0.000069,0.000000,0.000098,0.000089


In [160]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(df, test_size = 0.2)

In [161]:
train.index

Index(['TCGA-25-1635', 'TCGA-29-1691', 'TCGA-23-1030', 'TCGA-24-1565',
       'TCGA-24-2280', 'TCGA-10-0931', 'TCGA-24-2024', 'TCGA-10-0927',
       'TCGA-13-1497', 'TCGA-23-2078',
       ...
       'TCGA-24-2036', 'TCGA-04-1362', 'TCGA-13-0727', 'TCGA-25-2409',
       'TCGA-36-1570', 'TCGA-13-1487', 'TCGA-61-1736', 'TCGA-23-1023',
       'TCGA-24-1105', 'TCGA-04-1365'],
      dtype='object', name='PATIENT_ID', length=168)

In [162]:
trn_X_pd = train.drop(["PLATINUM_STATUS","label"],axis=1).values
trn_y_pd = train.label.values

print("Information of Training Data")
print("Samples : {}".format(trn_X_pd.shape[0]))
print("Features : {}".format(trn_X_pd.shape[1]))

Information of Training Data
Samples : 168
Features : 300


In [163]:
val_X_pd = val.drop(["PLATINUM_STATUS","label"],axis=1).values
val_y_pd = val.label.values

print("Information of Training Data")
print("Samples : {}".format(val_X_pd.shape[0]))
print("Features : {}".format(val_X_pd.shape[1]))

Information of Training Data
Samples : 42
Features : 300


In [164]:
df_label

,PLATINUM_STATUS,label
PATIENT_ID,,
TCGA-04-1331,Sensitive,0.0
TCGA-04-1332,Sensitive,0.0
TCGA-04-1347,Sensitive,0.0
TCGA-04-1362,Resistant,1.0
TCGA-04-1364,Resistant,1.0
...,...,...
TCGA-61-2098,Sensitive,0.0
TCGA-61-2109,Sensitive,0.0
TCGA-61-2110,Resistant,1.0


In [165]:
trn_X = torch.from_numpy(trn_X_pd.astype(float))
trn_y = torch.from_numpy(trn_y_pd.astype(float))

val_X = torch.from_numpy(val_X_pd.astype(float))
val_y = torch.from_numpy(val_y_pd.astype(float))

In [166]:
trn = Dataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True, drop_last = True)

trn_test_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=False)


val = Dataset(val_X, val_y)
test_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)

In [146]:
## Loss function
criterion = nn.BCELoss()
learning_rate = 1e-3
    
## After just second, will modify optimizer(weight_decay,step_size, base_lr, max_lr,scheduler)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay = 1e-5)
    
#model = DNN_seq_2()
use_cuda = torch.cuda.is_available()
num_epochs = 90
    
if use_cuda:
    model = model.cuda()
    
##############################################################################
step_size = 2000
base_lr, max_lr = 0.001, 0.01  
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                             lr=max_lr)
    
scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, step_size=step_size, mode='exp_range', gamma=0.99994)    

In [147]:


predict = []

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    avg_loss = 0.
    #correct = 0.   # Accuracy
    correct_val = 0.
        
    for batch_idx, trn in enumerate(trn_loader):
        trn_X, trn_y = trn['X'], trn['y']
        if use_cuda:
            trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
        trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
        optimizer.zero_grad()
        trn_pred = model(trn_X)
        for i in trn_pred:
            predict.append(i.detach().numpy())
            
        if scheduler:
            #print('cycle_LR')
            scheduler.batch_step()
        #print(trn_pred.squeeze())
        #print(trn_y)
        trn_loss = criterion(trn_pred.squeeze(), trn_y)
        trn_loss.backward()
        optimizer.step()
        
        avg_loss += trn_loss.item()/len(trn_loader)
        elapsed_time = time.time() - start_time
        predicted = (trn_pred >= 0.5).flatten().type(torch.ByteTensor)
        correct_val += (predicted == trn_y).sum()
    print('*'*20, 'Train', '*'*20)
    print('Loss: {}, Accuracy: {} % \t time={:.2f}s'.format(trn_loss.item(), correct_val/(len(trn_loader)*batch_size), elapsed_time))
print('*'*46)
print("Trn accuracy:{:.3f} ".format(float(correct_val) / (len(trn_loader)*batch_size)))

******************** Train ********************
Loss: 1.0071132183074951, Accuracy: 0.581818163394928 % 	 time=0.33s
******************** Train ********************
Loss: 0.41691604256629944, Accuracy: 0.7090908885002136 % 	 time=0.26s
******************** Train ********************
Loss: 0.7919425964355469, Accuracy: 0.6969696879386902 % 	 time=0.26s
******************** Train ********************
Loss: 0.545437216758728, Accuracy: 0.739393949508667 % 	 time=0.25s
******************** Train ********************
Loss: 0.34149715304374695, Accuracy: 0.7212121486663818 % 	 time=0.27s
******************** Train ********************
Loss: 1.0561163425445557, Accuracy: 0.7818182110786438 % 	 time=0.34s
******************** Train ********************
Loss: 0.39828401803970337, Accuracy: 0.7636363506317139 % 	 time=0.26s
******************** Train ********************
Loss: 0.3370503783226013, Accuracy: 0.7818182110786438 % 	 time=0.26s
******************** Train ********************
Loss: 1.

******************** Train ********************
Loss: 0.319019079208374, Accuracy: 0.8363636136054993 % 	 time=0.24s
******************** Train ********************
Loss: 0.2018052637577057, Accuracy: 0.8484848737716675 % 	 time=0.25s
******************** Train ********************
Loss: 0.11483678966760635, Accuracy: 0.8606060743331909 % 	 time=0.25s
******************** Train ********************
Loss: 0.15633618831634521, Accuracy: 0.8363636136054993 % 	 time=0.24s
******************** Train ********************
Loss: 0.3994195759296417, Accuracy: 0.8909090757369995 % 	 time=0.25s
******************** Train ********************
Loss: 0.5272725224494934, Accuracy: 0.8787878751754761 % 	 time=0.24s
******************** Train ********************
Loss: 0.281921923160553, Accuracy: 0.8606060743331909 % 	 time=0.26s
******************** Train ********************
Loss: 0.5282734632492065, Accuracy: 0.842424213886261 % 	 time=0.26s
******************** Train ********************
Loss: 0.3

In [153]:
model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(trn_test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(trn_test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(trn_test_loader)*batch_size)))

tensor([[0.2319],
        [0.6075],
        [0.0560],
        [0.0439],
        [0.0374]])
tensor([0, 1, 0, 0, 0], dtype=torch.uint8)
******************** Test ********************
Loss: 0.18055102229118347, Accuracy: 1.0 %
**********************************************
Val accuracy:1.000


In [154]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]
#trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
pd.DataFrame(trn_y_pd, predict_.iloc[:,0])

ValueError: Shape of passed values is (168, 1), indices imply (5, 1)

In [155]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(trn_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print('AUC: ', auc(fpr, trp))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [167]:
model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        #print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        print(val_y)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(test_loader)*batch_size)))

tensor([1, 0, 0, 0, 0], dtype=torch.uint8)
tensor([1., 0., 0., 0., 0.])
tensor([1, 0, 0, 0, 0], dtype=torch.uint8)
tensor([1., 0., 0., 0., 0.])
tensor([0, 0, 0, 0, 1], dtype=torch.uint8)
tensor([0., 0., 0., 0., 1.])
tensor([0, 0, 0, 1, 0], dtype=torch.uint8)
tensor([0., 0., 0., 1., 0.])
tensor([0, 1, 1, 0, 0], dtype=torch.uint8)
tensor([0., 1., 1., 0., 0.])
tensor([0, 1, 0, 0, 1], dtype=torch.uint8)
tensor([0., 1., 0., 0., 1.])
tensor([0, 0, 1, 0, 0], dtype=torch.uint8)
tensor([0., 0., 1., 0., 0.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([1., 0., 0., 0., 0.])
tensor([0, 0], dtype=torch.uint8)
tensor([0., 0.])
******************** Test ********************
Loss: 0.08248241990804672, Accuracy: 0.9111111164093018 %
**********************************************
Val accuracy:0.911


In [168]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
0.579014,1.0
0.035489,0.0
0.040849,0.0
0.022445,0.0
0.491972,0.0
0.537468,1.0
0.032577,0.0
0.050847,0.0
0.061795,0.0


In [169]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(val_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print(type(fpr))
#print(fpr)
print('AUC: ', auc(fpr, trp))

[[ 9  1]
 [ 0 32]]
Accuracy :  0.9761904761904762
Sensitivity :  0.9
Specificity :  1.0
<class 'numpy.ndarray'>
AUC:  0.95


In [170]:
torch.save({
#'epoch': EPOCHS,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': trn_loss
    }, "./platin_model_save/platin_model_200_|_100_model_2_lnc_transfer.pth")     

In [151]:
#model = DNN_seq()

use_cuda = torch.cuda.is_available()

if use_cuda:
    model = model.cuda()
    
fit(model, trn_loader)

Epoch : 0 [0/171 (0%)]	Loss: 0.746858	 Accuracy:50.000%
Epoch : 0 [32/171 (18%)]	Loss: 0.662493	 Accuracy:60.417%
Epoch : 0 [64/171 (36%)]	Loss: 0.704260	 Accuracy:60.000%
Epoch : 0 [96/171 (55%)]	Loss: 0.674394	 Accuracy:66.071%
Epoch : 0 [128/171 (73%)]	Loss: 0.722818	 Accuracy:67.361%
Epoch : 0 [110/171 (91%)]	Loss: 0.599336	 Accuracy:65.909%
Epoch : 1 [0/171 (0%)]	Loss: 0.575366	 Accuracy:87.500%
Epoch : 1 [32/171 (18%)]	Loss: 0.667310	 Accuracy:70.833%
Epoch : 1 [64/171 (36%)]	Loss: 0.673750	 Accuracy:68.750%
Epoch : 1 [96/171 (55%)]	Loss: 0.751240	 Accuracy:67.857%
Epoch : 1 [128/171 (73%)]	Loss: 0.740712	 Accuracy:67.361%
Epoch : 1 [110/171 (91%)]	Loss: 0.710288	 Accuracy:65.909%
Epoch : 2 [0/171 (0%)]	Loss: 0.641428	 Accuracy:62.500%
Epoch : 2 [32/171 (18%)]	Loss: 0.618767	 Accuracy:68.750%
Epoch : 2 [64/171 (36%)]	Loss: 0.619703	 Accuracy:65.000%
Epoch : 2 [96/171 (55%)]	Loss: 0.579154	 Accuracy:65.179%
Epoch : 2 [128/171 (73%)]	Loss: 0.667362	 Accuracy:66.667%
Epoch : 2 [110/

In [152]:
model.eval()
#predict = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        val_pred = val_pred.type_as(torch.FloatTensor())
        val_loss = criterion(val_pred, val_y)
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

******************** Test ********************
Loss: 0.5934396982192993, Accuracy: 0.7291666865348816 %
**********************************************
Val accuracy:0.729


# Voting

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifier
# VotingClassifier
from sklearn.ensemble import VotingClassifier

# ensemble 할 model 정의
models = [
    ('ada', AdaBoostClassifier()),
    ('bc', BaggingClassifier()),
    ('etc',ExtraTreesClassifier()),
    ('gbc', GradientBoostingClassifier()),
    ('rfc', RandomForestClassifier()),
    ('knn', KNeighborsClassifier()),
    ('svc', SVC(probability=True)),
    ('xgb', XGBClassifier()),
    ('lgbm', LGBMClassifier()),
    ('dtc', DecisionTreeClassifier()),
    ('lr', LogisticRegressionCV()),
    ('ridge', RidgeClassifier()),
]

# hard vote
hard_vote  = VotingClassifier(models, voting='hard')
hard_vote_cv = cross_validate(hard_vote, x_train, y_train, cv=k_fold)
hard_vote.fit(x_train, y_train)

# soft vote
soft_vote  = VotingClassifier(models, voting='soft')
soft_vote_cv = cross_validate(soft_vote, x_train, y_train, cv=k_fold)
soft_vote.fit(x_train, y_train)


In [52]:
trn_X = torch.from_numpy(trn_X_pd.astype(float))
trn_y = torch.from_numpy(trn_y_pd.astype(float))

val_X = torch.from_numpy(val_X_pd.astype(float))
val_y = torch.from_numpy(val_y_pd.astype(float))

In [53]:
trn = Dataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True, drop_last = True)

trn_test_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=False)


val = Dataset(val_X, val_y)
test_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)

## Model 1 (Voting)

In [93]:
## Loss function
criterion = nn.BCELoss()
learning_rate = 1e-3
model_1 = DNN_seq_1()   
## After just second, will modify optimizer(weight_decay,step_size, base_lr, max_lr,scheduler)
optimizer = optim.Adam(model_1.parameters(), lr=0.001, weight_decay = 1e-5)
    
model_1 = DNN_seq_1()


##
use_cuda = torch.cuda.is_available()
num_epochs = 90
    
if use_cuda:
    model_1 = model_1.cuda()
    
##############################################################################
step_size = 2000
base_lr, max_lr = 0.001, 0.01  
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model_1.parameters()), 
                             lr=max_lr)
    
scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, step_size=step_size, mode='exp_range', gamma=0.99994)    

In [94]:


predict = []

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    avg_loss = 0.
    #correct = 0.   # Accuracy
    correct_val = 0.
        
    for batch_idx, trn in enumerate(trn_loader):
        trn_X, trn_y = trn['X'], trn['y']
        if use_cuda:
            trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
        trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
        optimizer.zero_grad()
        trn_pred = model_1(trn_X)
        for i in trn_pred:
            predict.append(i.detach().numpy())
            
        if scheduler:
            #print('cycle_LR')
            scheduler.batch_step()
        #print(trn_pred.squeeze())
        #print(trn_y)
        trn_loss = criterion(trn_pred.squeeze(), trn_y)
        trn_loss.backward()
        optimizer.step()
        
        avg_loss += trn_loss.item()/len(trn_loader)
        elapsed_time = time.time() - start_time
        predicted = (trn_pred >= 0.5).flatten().type(torch.ByteTensor)
        correct_val += (predicted == trn_y).sum()
    print('*'*20, 'Train', '*'*20)
    print('Loss: {}, Accuracy: {} % \t time={:.2f}s'.format(trn_loss.item(), correct_val/(len(trn_loader)*batch_size), elapsed_time))
print('*'*46)
print("Trn accuracy:{:.3f} ".format(float(correct_val) / (len(trn_loader)*batch_size)))

******************** Train ********************
Loss: 0.62250155210495, Accuracy: 0.6727272868156433 % 	 time=0.16s
******************** Train ********************
Loss: 0.28187888860702515, Accuracy: 0.7757575511932373 % 	 time=0.14s
******************** Train ********************
Loss: 0.4212983250617981, Accuracy: 0.7818182110786438 % 	 time=0.15s
******************** Train ********************
Loss: 0.3047163784503937, Accuracy: 0.8242424130439758 % 	 time=0.14s
******************** Train ********************
Loss: 0.18727238476276398, Accuracy: 0.8727272748947144 % 	 time=0.19s
******************** Train ********************
Loss: 0.15723516047000885, Accuracy: 0.8727272748947144 % 	 time=0.17s
******************** Train ********************
Loss: 0.08779895305633545, Accuracy: 0.8666666746139526 % 	 time=0.15s
******************** Train ********************
Loss: 1.234900951385498, Accuracy: 0.8606060743331909 % 	 time=0.14s
******************** Train ********************
Loss: 0

******************** Train ********************
Loss: 0.025340717285871506, Accuracy: 0.939393937587738 % 	 time=0.14s
******************** Train ********************
Loss: 0.020182045176625252, Accuracy: 0.939393937587738 % 	 time=0.14s
******************** Train ********************
Loss: 0.00676519563421607, Accuracy: 0.9636363387107849 % 	 time=0.14s
******************** Train ********************
Loss: 0.06858143955469131, Accuracy: 0.9515151381492615 % 	 time=0.14s
******************** Train ********************
Loss: 0.03390727937221527, Accuracy: 0.9575757384300232 % 	 time=0.15s
******************** Train ********************
Loss: 0.00573237519711256, Accuracy: 0.9454545378684998 % 	 time=0.15s
******************** Train ********************
Loss: 0.009802227839827538, Accuracy: 0.9696969985961914 % 	 time=0.14s
******************** Train ********************
Loss: 0.006746111903339624, Accuracy: 0.9818181991577148 % 	 time=0.13s
******************** Train *******************

## Model 2 (Voting)

In [127]:
## Loss function
criterion = nn.BCELoss()
learning_rate = 1e-3
model_2 = DNN_seq_2()   
## After just second, will modify optimizer(weight_decay,step_size, base_lr, max_lr,scheduler)
optimizer = optim.Adam(model_2.parameters(), lr=0.001, weight_decay = 1e-5)
    
model_2 = DNN_seq_2()




use_cuda = torch.cuda.is_available()
num_epochs = 90
    
if use_cuda:
    model_2 = model_2.cuda()
    
##############################################################################
step_size = 2000
base_lr, max_lr = 0.001, 0.01  
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model_2.parameters()), 
                             lr=max_lr)
    
scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, step_size=step_size, mode='exp_range', gamma=0.99994)    

In [128]:


predict = []

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    avg_loss = 0.
    #correct = 0.   # Accuracy
    correct_val = 0.
        
    for batch_idx, trn in enumerate(trn_loader):
        trn_X, trn_y = trn['X'], trn['y']
        if use_cuda:
            trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
        trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
        optimizer.zero_grad()
        trn_pred = model_2(trn_X)
        for i in trn_pred:
            predict.append(i.detach().numpy())
            
        if scheduler:
            #print('cycle_LR')
            scheduler.batch_step()
        #print(trn_pred.squeeze())
        #print(trn_y)
        trn_loss = criterion(trn_pred.squeeze(), trn_y)
        trn_loss.backward()
        optimizer.step()
        
        avg_loss += trn_loss.item()/len(trn_loader)
        elapsed_time = time.time() - start_time
        predicted = (trn_pred >= 0.5).flatten().type(torch.ByteTensor)
        correct_val += (predicted == trn_y).sum()
    print('*'*20, 'Train', '*'*20)
    print('Loss: {}, Accuracy: {} % \t time={:.2f}s'.format(trn_loss.item(), correct_val/(len(trn_loader)*batch_size), elapsed_time))
print('*'*46)
print("Trn accuracy:{:.3f} ".format(float(correct_val) / (len(trn_loader)*batch_size)))

******************** Train ********************
Loss: 0.8399748802185059, Accuracy: 0.4727272689342499 % 	 time=0.32s
******************** Train ********************
Loss: 0.7234965562820435, Accuracy: 0.5333333611488342 % 	 time=0.26s
******************** Train ********************
Loss: 0.432243674993515, Accuracy: 0.4545454680919647 % 	 time=0.25s
******************** Train ********************
Loss: 0.6998279094696045, Accuracy: 0.5878787636756897 % 	 time=0.25s
******************** Train ********************
Loss: 0.5822752118110657, Accuracy: 0.6000000238418579 % 	 time=0.26s
******************** Train ********************
Loss: 0.6820012927055359, Accuracy: 0.6484848260879517 % 	 time=0.25s
******************** Train ********************
Loss: 0.456350713968277, Accuracy: 0.6060606241226196 % 	 time=0.25s
******************** Train ********************
Loss: 0.7419701814651489, Accuracy: 0.7272727489471436 % 	 time=0.26s
******************** Train ********************
Loss: 0.69

******************** Train ********************
Loss: 0.4875887930393219, Accuracy: 0.8606060743331909 % 	 time=0.28s
******************** Train ********************
Loss: 0.6442869901657104, Accuracy: 0.9090909361839294 % 	 time=0.25s
******************** Train ********************
Loss: 0.1508985310792923, Accuracy: 0.8969696760177612 % 	 time=0.25s
******************** Train ********************
Loss: 0.517178475856781, Accuracy: 0.9090909361839294 % 	 time=0.25s
******************** Train ********************
Loss: 0.39280542731285095, Accuracy: 0.842424213886261 % 	 time=0.25s
******************** Train ********************
Loss: 0.6134015321731567, Accuracy: 0.8666666746139526 % 	 time=0.25s
******************** Train ********************
Loss: 0.4274056851863861, Accuracy: 0.8666666746139526 % 	 time=0.25s
******************** Train ********************
Loss: 0.5373755693435669, Accuracy: 0.8787878751754761 % 	 time=0.25s
******************** Train ********************
Loss: 0.4

In [129]:
criterion = nn.BCELoss()

In [130]:
model_1.eval()
#predict = []
#y_val = val_y
predict_1 = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict_1.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        #print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        print(val_y)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(test_loader)*batch_size)))

tensor([1, 1, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 0., 1.])
tensor([0, 0, 1, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 0., 0.])
tensor([1, 0, 1, 0, 0], dtype=torch.uint8)
tensor([0., 0., 1., 0., 0.])
tensor([0, 0, 1, 0, 0], dtype=torch.uint8)
tensor([1., 0., 1., 0., 0.])
tensor([0, 1, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 1., 0.])
tensor([0, 0, 0, 1, 1], dtype=torch.uint8)
tensor([0., 0., 0., 0., 1.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 0., 0.])
tensor([0, 1, 1, 0, 1], dtype=torch.uint8)
tensor([0., 0., 1., 0., 1.])
tensor([1, 0], dtype=torch.uint8)
tensor([1., 1.])
******************** Test ********************
Loss: 11.427840232849121, Accuracy: 0.6888889074325562 %
**********************************************
Val accuracy:0.689


In [131]:
predict_ = pd.DataFrame(predict_1)

predict_.iloc[:,0]
#trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
9.805429e-01,0.0
9.976170e-01,1.0
4.016732e-05,0.0
1.509615e-10,0.0
1.157224e-12,1.0
2.218026e-07,0.0
2.021635e-08,1.0
9.078228e-01,0.0
4.676259e-02,0.0


In [132]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict_1)

predict_.iloc[:,0]

label = pd.DataFrame(val_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print('AUC: ', auc(fpr, trp))

[[ 8  6]
 [ 5 23]]
Accuracy :  0.7380952380952381
Sensitivity :  0.5714285714285714
Specificity :  0.8214285714285714
AUC:  0.6964285714285714


In [133]:
criterion = nn.BCELoss()

In [134]:
model_2.eval()
#predict = []
#y_val = val_y
predict_2 = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict_2.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        #print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        print(val_y)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(test_loader)*batch_size)))

tensor([1, 1, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 0., 1.])
tensor([0, 0, 0, 1, 0], dtype=torch.uint8)
tensor([0., 1., 0., 0., 0.])
tensor([0, 0, 1, 0, 0], dtype=torch.uint8)
tensor([0., 0., 1., 0., 0.])
tensor([0, 0, 1, 0, 0], dtype=torch.uint8)
tensor([1., 0., 1., 0., 0.])
tensor([0, 1, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 1., 0.])
tensor([0, 0, 0, 1, 1], dtype=torch.uint8)
tensor([0., 0., 0., 0., 1.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 0., 0.])
tensor([0, 1, 1, 0, 1], dtype=torch.uint8)
tensor([0., 0., 1., 0., 1.])
tensor([0, 0], dtype=torch.uint8)
tensor([1., 1.])
******************** Test ********************
Loss: 12.315314292907715, Accuracy: 0.6888889074325562 %
**********************************************
Val accuracy:0.689


In [135]:
predict_ = pd.DataFrame(predict_2)

predict_.iloc[:,0]
#trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
9.689116e-01,0.0
9.968559e-01,1.0
7.458268e-05,0.0
7.186585e-14,0.0
2.359037e-08,1.0
2.534286e-06,0.0
1.988177e-04,1.0
3.421953e-02,0.0
9.593546e-01,0.0


In [136]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict_2)

predict_.iloc[:,0]

label = pd.DataFrame(val_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print('AUC: ', auc(fpr, trp))

[[ 7  7]
 [ 4 24]]
Accuracy :  0.7380952380952381
Sensitivity :  0.5
Specificity :  0.8571428571428571
AUC:  0.6785714285714286


In [137]:
predict_1

[array([0.9805429], dtype=float32),
 array([0.997617], dtype=float32),
 array([4.0167324e-05], dtype=float32),
 array([1.5096147e-10], dtype=float32),
 array([1.1572244e-12], dtype=float32),
 array([2.2180257e-07], dtype=float32),
 array([2.0216353e-08], dtype=float32),
 array([0.90782285], dtype=float32),
 array([0.04676259], dtype=float32),
 array([3.9649746e-07], dtype=float32),
 array([0.80628973], dtype=float32),
 array([1.897543e-08], dtype=float32),
 array([0.999223], dtype=float32),
 array([1.1756502e-05], dtype=float32),
 array([4.237735e-11], dtype=float32),
 array([0.24171144], dtype=float32),
 array([3.6049649e-12], dtype=float32),
 array([0.9923767], dtype=float32),
 array([0.00096397], dtype=float32),
 array([4.6184997e-10], dtype=float32),
 array([2.5939459e-19], dtype=float32),
 array([0.99906236], dtype=float32),
 array([4.1128143e-05], dtype=float32),
 array([0.1152596], dtype=float32),
 array([0.13516602], dtype=float32),
 array([5.560944e-13], dtype=float32),
 array

In [138]:
predict_2

[array([0.9689116], dtype=float32),
 array([0.99685585], dtype=float32),
 array([7.4582684e-05], dtype=float32),
 array([7.186585e-14], dtype=float32),
 array([2.3590365e-08], dtype=float32),
 array([2.5342856e-06], dtype=float32),
 array([0.00019882], dtype=float32),
 array([0.03421953], dtype=float32),
 array([0.9593546], dtype=float32),
 array([5.231794e-12], dtype=float32),
 array([0.49860108], dtype=float32),
 array([2.235494e-08], dtype=float32),
 array([0.9998287], dtype=float32),
 array([0.00053509], dtype=float32),
 array([6.6206e-12], dtype=float32),
 array([0.00229861], dtype=float32),
 array([4.1592455e-05], dtype=float32),
 array([0.9989016], dtype=float32),
 array([0.04136225], dtype=float32),
 array([1.0710325e-14], dtype=float32),
 array([7.562101e-12], dtype=float32),
 array([0.9999052], dtype=float32),
 array([2.216958e-07], dtype=float32),
 array([0.02295523], dtype=float32),
 array([0.02267374], dtype=float32),
 array([4.084518e-15], dtype=float32),
 array([2.182266

In [139]:
#predict_3 = predict_1 + predict_2

In [143]:
len(predict_3)

TypeError: object of type 'map' has no len()

In [144]:
type(predict_1)

list

In [145]:
import operator

predict_3 = map(operator.add, predict_1, predict_2)

In [146]:
print(predict_3)

In [147]:
predict = []

for i in predict_3:
    print (i/2)
    predict.append(i/2)
    print (predict)



[0.9747273]
[array([0.9747273], dtype=float32)]
[0.99723643]
[array([0.9747273], dtype=float32), array([0.99723643], dtype=float32)]
[5.7375004e-05]
[array([0.9747273], dtype=float32), array([0.99723643], dtype=float32), array([5.7375004e-05], dtype=float32)]
[7.551666e-11]
[array([0.9747273], dtype=float32), array([0.99723643], dtype=float32), array([5.7375004e-05], dtype=float32), array([7.551666e-11], dtype=float32)]
[1.1795761e-08]
[array([0.9747273], dtype=float32), array([0.99723643], dtype=float32), array([5.7375004e-05], dtype=float32), array([7.551666e-11], dtype=float32), array([1.1795761e-08], dtype=float32)]
[1.3780441e-06]
[array([0.9747273], dtype=float32), array([0.99723643], dtype=float32), array([5.7375004e-05], dtype=float32), array([7.551666e-11], dtype=float32), array([1.1795761e-08], dtype=float32), array([1.3780441e-06], dtype=float32)]
[9.9418954e-05]
[array([0.9747273], dtype=float32), array([0.99723643], dtype=float32), array([5.7375004e-05], dtype=float32), ar

In [148]:
print(predict)
type(predict)

[array([0.9747273], dtype=float32), array([0.99723643], dtype=float32), array([5.7375004e-05], dtype=float32), array([7.551666e-11], dtype=float32), array([1.1795761e-08], dtype=float32), array([1.3780441e-06], dtype=float32), array([9.9418954e-05], dtype=float32), array([0.47102118], dtype=float32), array([0.50305855], dtype=float32), array([1.9825134e-07], dtype=float32), array([0.65244544], dtype=float32), array([2.0665183e-08], dtype=float32), array([0.99952585], dtype=float32), array([0.00027342], dtype=float32), array([2.4498976e-11], dtype=float32), array([0.12200502], dtype=float32), array([2.079623e-05], dtype=float32), array([0.99563915], dtype=float32), array([0.02116311], dtype=float32), array([2.3093034e-10], dtype=float32), array([3.7810506e-12], dtype=float32), array([0.9994838], dtype=float32), array([2.067492e-05], dtype=float32), array([0.06910741], dtype=float32), array([0.07891988], dtype=float32), array([2.8008946e-13], dtype=float32), array([2.1300366e-07], dtype=

list

In [149]:
len(trn_y_pd)

168

## Evalutation (Voting)

In [150]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]
#trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
9.747273e-01,0.0
9.972364e-01,1.0
5.737500e-05,0.0
7.551666e-11,0.0
1.179576e-08,1.0
1.378044e-06,0.0
9.941895e-05,1.0
4.710212e-01,0.0
5.030586e-01,0.0


In [151]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(val_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print('AUC: ', auc(fpr, trp))

[[ 7  7]
 [ 5 23]]
Accuracy :  0.7142857142857143
Sensitivity :  0.5
Specificity :  0.8214285714285714
AUC:  0.6607142857142857


In [104]:
val_y_pd_ = val_y_pd.to_numpy()
val_y_pd_ = torch.from_numpy(val_y_pd_)
val_y_pd_ = val_y_pd_.type_as(torch.FloatTensor())

In [ ]:
val_loss = criterion(predict, val_y)
val_loss_summary += val_loss
predicted = torch.max(val_pred.data, 1)[1] 
correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

In [96]:
val_y_pd

0     0.0
1     0.0
2     0.0
3     0.0
4     1.0
5     0.0
6     0.0
7     0.0
8     1.0
9     0.0
10    0.0
11    0.0
12    0.0
13    1.0
14    0.0
15    0.0
16    0.0
17    1.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    1.0
27    0.0
28    0.0
29    1.0
30    0.0
31    0.0
32    0.0
33    0.0
34    0.0
35    0.0
36    1.0
37    1.0
38    0.0
39    0.0
40    0.0
41    0.0
42    0.0
Name: label, dtype: float64

In [93]:
#num_ftrs = model.forward.in_features
 
for name, param in model.named_parameters():
    print (name, param.data)

layer1.0.weight tensor([[-0.0565, -0.1205,  0.0296,  ...,  0.2230, -0.0025,  0.0114],
        [-0.0879, -0.0202, -0.0235,  ...,  0.0587,  0.0370, -0.0111],
        [-0.0381, -0.0511, -0.0141,  ..., -0.0142,  0.0329,  0.0550],
        ...,
        [ 0.0349, -0.0490, -0.0410,  ..., -0.0243, -0.0495,  0.0048],
        [ 0.0487,  0.1016, -0.0085,  ..., -0.1036,  0.0225, -0.0505],
        [-0.0129,  0.1087,  0.0454,  ..., -0.1197,  0.0375,  0.0101]])
layer1.0.bias tensor([-0.0296, -0.0339,  0.0066, -0.0166, -0.0347, -0.0489,  0.0233,  0.0351,
         0.0348,  0.0116,  0.0404, -0.0197, -0.0154,  0.0126,  0.0236,  0.0361,
        -0.0178, -0.0001, -0.0028, -0.0274, -0.0495, -0.0317,  0.0020,  0.0040,
         0.0019,  0.0077,  0.0134, -0.0460,  0.0494,  0.0400, -0.0072, -0.0119,
         0.0044, -0.0115,  0.0237, -0.0051, -0.0136, -0.0101,  0.0195,  0.0419,
        -0.0333, -0.0286,  0.0231, -0.0157,  0.0131,  0.0034, -0.0038,  0.0113,
        -0.0342,  0.0022, -0.0403,  0.0045,  0.0368, -0.

In [95]:
model.layer1 = 

NameError: name 'Sequential' is not defined

In [79]:
nn.Linear(num_ftrs)

TypeError: __init__() missing 1 required positional argument: 'out_features'

In [32]:
list(model.children())[0:3]

AttributeError: 'list' object has no attribute 'parameter'

## Train model

In [77]:
model = MLPRegressor()

NameError: name 'X_features' is not defined

In [78]:
criterion = nn.MSELoss()
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

NameError: name 'model' is not defined

In [44]:
batch_size = 32

In [45]:
trn = data_utils.TensorDataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)

val = data_utils.TensorDataset(val_X, val_y)
val_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)

NameError: name 'val_X' is not defined

In [13]:
trn_X_pd = df2_mod.iloc[:,2:]

In [14]:
trn_X_pd

ID,ENSG00000069482,ENSG00000072657,ENSG00000078399,ENSG00000080572,ENSG00000100678,ENSG00000104435,ENSG00000104888,ENSG00000105146,ENSG00000109321,ENSG00000112499,...,ENSG00000274576,ENSG00000275756,ENSG00000276775,ENSG00000277247,ENSG00000278196,ENSG00000278698,ENSG00000279834,ENSG00000279970,ENSG00000280411,ENSG00000281880
TCGA-61-1910,0.970585,0.003561,0.044476,0.000000,0.000000,0.000000,0.043732,2.308592,1.014494,0.000000,...,0.000000,0.920597,0.000000,1.604301,0.299376,0.000000,0.894232,1.612154,0.761720,0.000000
TCGA-24-0982,0.395276,0.005010,0.090384,0.007062,0.000000,0.006438,0.082034,0.242661,0.691196,0.004156,...,24.797152,0.259035,4.625172,0.188089,4.211878,0.270091,0.034947,0.311310,0.357217,0.000000
TCGA-36-1580,1.403594,0.008641,0.133845,0.001970,0.002559,0.041762,0.074384,0.984290,2.329158,0.001391,...,4.017149,0.289083,0.893372,0.314861,4.817979,0.000000,0.000000,0.707253,15.996040,0.000000
TCGA-25-1321,0.308600,0.003129,0.004478,0.162223,0.000788,0.000000,0.101272,0.240456,1.740308,0.004819,...,2.576079,0.166843,0.000000,0.121147,1.356422,0.521892,0.157563,2.033770,1.495528,0.000000
TCGA-30-1892,0.206074,0.133216,0.119614,0.056072,0.000000,0.055381,0.280458,0.592872,0.914734,0.004400,...,1.058604,0.114269,2.825069,0.000000,3.716018,0.000000,0.000000,0.294278,1.890974,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-04-1347,0.319288,0.000491,0.033696,0.005704,0.000988,0.011701,0.060742,0.663423,0.094262,0.000671,...,0.000000,0.000000,0.000000,3.038608,0.567030,0.000000,0.000000,0.071847,4.616723,0.000000
TCGA-24-0968,0.237050,0.009708,0.185222,0.000000,0.000000,0.000000,0.307004,0.213121,1.291373,0.003796,...,15.221570,0.525782,4.332951,2.720169,31.845646,0.205584,0.000000,0.710875,6.117782,0.000000
TCGA-30-1891,0.772034,0.000481,0.110194,0.000000,0.000646,0.200678,0.000000,0.098616,2.959271,0.077290,...,47.331214,0.000000,2.631492,0.000000,65.568652,0.000000,0.000000,0.000000,80.395413,0.000000
TCGA-09-0366,0.195435,0.000085,1.631170,0.026487,0.001376,0.006339,0.303848,0.703636,5.424154,0.008418,...,0.225023,0.534375,0.100086,1.481522,3.712525,0.000000,0.058985,2.477118,0.602935,0.001573


In [15]:
trn_y_pd = df2_mod.iloc[:,1]

In [16]:
trn_y_pd

TCGA-61-1910    0.007666
TCGA-24-0982    0.007190
TCGA-36-1580    0.027081
TCGA-25-1321    0.002315
TCGA-30-1892    0.057091
                  ...   
TCGA-04-1347    0.000000
TCGA-24-0968    0.013682
TCGA-30-1891    0.034186
TCGA-09-0366    0.032361
TCGA-13-0884    0.018597
Name: ENSG00000048545, Length: 214, dtype: float64

In [17]:
trn_X = torch.from_numpy(trn_X_pd.astype(float).values)
trn_y = torch.from_numpy(trn_y_pd.astype(float).values)

In [18]:
batch_size=64

In [19]:
trn = data_utils.TensorDataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)

In [20]:
tmp = next(iter(trn_loader))

In [21]:
# for dictionary batch
class Dataset(data_utils.Dataset):
   
    def __init__(self, X, y):
        self.X = X
        self.y = y
   
    def __getitem__(self, idx):
        return {'X': self.X[idx], 'y': self.y[idx]}
   
    def __len__(self):
        return len(self.X)

In [22]:
trn = Dataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)